In [ ]:
pip install snscrape
pip install transformers
pip install reverse-geocode #lat-long to country
pip install pycountry #country to iso_code
pip install flashgeotext

In [ ]:
pip install geograpy3

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import requests
import json
import reverse_geocode
import pycountry
from flashgeotext.geotext import GeoText

geotext = GeoText()

In [ ]:
year = "2020"

In [ ]:
point = (-74.041878, 40.570842)

def check_country(point):
  # check to see which country it contains the point
  if point:
    point = point, #(lat,long)
    return reverse_geocode.search(point)[0]['country']
  return None          

print(check_country(point))

French Southern Territories


In [ ]:
point = (-74.041878, 40.570842)

def check_code(point):
  # check to see which country it contains the point
  if point:
    point = point, #(lat,long)
    return reverse_geocode.search(point)[0]['country_code']
  return None          

print(check_code(point))

TF


In [ ]:
# import geograpy

def find_country(text):
  try:
    return list(geotext.extract(input_text=text)['countries'].keys())[0]
  except:
    return None  
  # return places.country_mentions[0][0] if places.country_mentions is not None else None

In [ ]:
# Creating list to append tweet data to
tweets_list = []

# Using TwitterSearchScraper to scrape data and append tweets to list
query = "Dogecoin min_faves:100 since:2021-01-01 until:2021-12-31"
for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
  tweets_list.append([tweet.date,tweet.rawContent,tweet.user.location,tweet.place,
                      check_country((tweet.coordinates.latitude,tweet.coordinates.longitude) if tweet.coordinates else None),
                      check_code((tweet.coordinates.latitude,tweet.coordinates.longitude) if tweet.coordinates else None)])


In [ ]:
tweets_list

In [ ]:
tweets_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet','User Location','Place','Tweet Location','Country Code'])
tweets_df['Date'] = pd.to_datetime(tweets_df['Datetime']).dt.date
tweets_df['Year'] = tweets_df['Date'].apply(lambda x:x.year)

In [ ]:
# tweets_df.to_csv('doge_2021-01-01_2021-12-31.csv')

In [ ]:
countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3
    countries[country.alpha_2] = country.alpha_3
    try:
      countries[country.official_name] = country.alpha_3
    except:
      pass  

In [ ]:
# tweets_df = pd.read_csv("/content/doge_2021-01-01_2021-12-31.csv")

In [ ]:
# codes = [countries.get(country, 'Unknown code') for country in input_countries]
tweets_df['Country Code'] = tweets_df['Country Code'].apply(lambda x: countries.get(x, None))

In [ ]:
tweets_df['User Location'] = tweets_df['User Location'].apply(lambda x: None if x == "" else x)

In [ ]:
tweets_df['User Location'] = tweets_df['User Location'].apply(lambda x: find_country(x) if x != None else None)

In [ ]:
tweets_df

In [ ]:
tweets_df['Country Code'] = tweets_df.apply(lambda x: countries.get(x['User Location'], None) if pd.isnull(x['Country Code']) else x['Country Code'],axis=1)

In [ ]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25277 entries, 0 to 25276
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      25277 non-null  int64 
 1   Datetime        25277 non-null  object
 2   Tweet           25277 non-null  object
 3   User Location   3676 non-null   object
 4   Place           182 non-null    object
 5   Tweet Location  182 non-null    object
 6   Country Code    3815 non-null   object
 7   Date            25277 non-null  object
 8   Year            25277 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 1.7+ MB


In [ ]:
# tweets_df.to_csv('doge_2021-01-01_2021-12-31_country_code.csv')

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# find prediction
def prediction(text):
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  return scores[0],scores[1],scores[2]

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
# MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

In [ ]:
import pandas as pd
tweets_df = pd.read_csv("/content/doge_2021-01-01_2021-12-31_country_code.csv")

In [ ]:

tweets_df.dropna(subset=['Country Code'],inplace=True)

In [ ]:
tweets_df

,Unnamed: 0.1,Unnamed: 0,Datetime,Tweet,User Location,Place,Tweet Location,Country Code,Date,Year
0,0,0,2021-12-30 23:41:36+00:00,#dogecoin back to $1 in no time.,United States,NaN,NaN,USA,2021-12-30,2021
7,7,7,2021-12-30 20:56:54+00:00,$Doge building MORE energy and $COC ALREADY UP...,United States,NaN,NaN,USA,2021-12-30,2021
8,8,8,2021-12-30 18:25:30+00:00,🌕 #Dogecoin $17\n ...,United States,NaN,NaN,USA,2021-12-30,2021
9,9,9,2021-12-30 18:10:28+00:00,Fun fact: The highest paid female athlete on t...,United States,NaN,NaN,USA,2021-12-30,2021
10,10,10,2021-12-30 17:42:20+00:00,Dogecoin,United States,NaN,NaN,USA,2021-12-30,2021
...,...,...,...,...,...,...,...,...,...,...
25226,25226,25226,2021-01-02 18:31:29+00:00,dogecoin revolution has begun \n\n(this is not...,NaN,"Place(id='01a9a39529b27f36', fullName='Manhatt...",United States,USA,2021-01-02,2021
25246,25246,25246,2021-01-02 14:21:39+00:00,buy and hold dogecoin should be 3 in marketcap...,United States,NaN,NaN,USA,2021-01-02,2021
25250,25250,25250,2021-01-02 06:33:17+00:00,Join https://t.co/WtMANwe41B today and Avoid n...,Kenya,NaN,NaN,KEN,2021-01-02,2021
25254,25254,25254,2021-01-02 05:16:39+00:00,@elonmusk #dogecoin maybe 2021 is the year Dog...,United States,NaN,NaN,USA,2021-01-02,2021


In [ ]:
tweets_df['Sentiment'] = tweets_df['Tweet'].apply(prediction)
tweets_df[['Negative','Neutral','Positive']] = pd.DataFrame(tweets_df['Sentiment'].tolist(), index=tweets_df.index)
# tweets_df['Preprocessed'] = tweets_df['Content'].apply(preprocess)

In [ ]:
tweets_df

,Unnamed: 0.1,Unnamed: 0,Datetime,Tweet,User Location,Place,Tweet Location,Country Code,Date,Year,Sentiment,Negative,Neutral,Positive
0,0,0,2021-12-30 23:41:36+00:00,#dogecoin back to $1 in no time.,United States,NaN,NaN,USA,2021-12-30,2021,"(0.0047556153, 0.13623151, 0.8590129)",0.004756,0.136232,0.859013
7,7,7,2021-12-30 20:56:54+00:00,$Doge building MORE energy and $COC ALREADY UP...,United States,NaN,NaN,USA,2021-12-30,2021,"(0.0021557633, 0.024547586, 0.97329664)",0.002156,0.024548,0.973297
8,8,8,2021-12-30 18:25:30+00:00,🌕 #Dogecoin $17\n ...,United States,NaN,NaN,USA,2021-12-30,2021,"(0.021169387, 0.7398118, 0.2390189)",0.021169,0.739812,0.239019
9,9,9,2021-12-30 18:10:28+00:00,Fun fact: The highest paid female athlete on t...,United States,NaN,NaN,USA,2021-12-30,2021,"(0.0035480494, 0.14258584, 0.85386616)",0.003548,0.142586,0.853866
10,10,10,2021-12-30 17:42:20+00:00,Dogecoin,United States,NaN,NaN,USA,2021-12-30,2021,"(0.051605787, 0.7534284, 0.19496582)",0.051606,0.753428,0.194966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25226,25226,25226,2021-01-02 18:31:29+00:00,dogecoin revolution has begun \n\n(this is not...,NaN,"Place(id='01a9a39529b27f36', fullName='Manhatt...",United States,USA,2021-01-02,2021,"(0.04810918, 0.6466202, 0.30527058)",0.048109,0.646620,0.305271
25246,25246,25246,2021-01-02 14:21:39+00:00,buy and hold dogecoin should be 3 in marketcap...,United States,NaN,NaN,USA,2021-01-02,2021,"(0.008810052, 0.17036851, 0.82082146)",0.008810,0.170369,0.820821
25250,25250,25250,2021-01-02 06:33:17+00:00,Join https://t.co/WtMANwe41B today and Avoid n...,Kenya,NaN,NaN,KEN,2021-01-02,2021,"(0.005783117, 0.23608427, 0.7581326)",0.005783,0.236084,0.758133
25254,25254,25254,2021-01-02 05:16:39+00:00,@elonmusk #dogecoin maybe 2021 is the year Dog...,United States,NaN,NaN,USA,2021-01-02,2021,"(0.0027663063, 0.2815225, 0.7157112)",0.002766,0.281523,0.715711


In [ ]:
tweets_df['Negative_Count'] = tweets_df['Negative'].apply(lambda x: 1 if x > 0.5 else 0)
tweets_df['Neutral_Count'] = tweets_df['Neutral'].apply(lambda x: 1 if x > 0.5 else 0)
tweets_df['Positive_Count'] = tweets_df['Positive'].apply(lambda x: 1 if x > 0.5 else 0)

In [ ]:
tweets_df

,Unnamed: 0.1,Unnamed: 0,Datetime,Tweet,User Location,Place,Tweet Location,Country Code,Date,Year,Sentiment,Negative,Neutral,Positive,Negative_Count,Neutral_Count,Positive_Count
0,0,0,2021-12-30 23:41:36+00:00,#dogecoin back to $1 in no time.,United States,NaN,NaN,USA,2021-12-30,2021,"(0.0047556153, 0.13623151, 0.8590129)",0.004756,0.136232,0.859013,0,0,1
7,7,7,2021-12-30 20:56:54+00:00,$Doge building MORE energy and $COC ALREADY UP...,United States,NaN,NaN,USA,2021-12-30,2021,"(0.0021557633, 0.024547586, 0.97329664)",0.002156,0.024548,0.973297,0,0,1
8,8,8,2021-12-30 18:25:30+00:00,🌕 #Dogecoin $17\n ...,United States,NaN,NaN,USA,2021-12-30,2021,"(0.021169387, 0.7398118, 0.2390189)",0.021169,0.739812,0.239019,0,1,0
9,9,9,2021-12-30 18:10:28+00:00,Fun fact: The highest paid female athlete on t...,United States,NaN,NaN,USA,2021-12-30,2021,"(0.0035480494, 0.14258584, 0.85386616)",0.003548,0.142586,0.853866,0,0,1
10,10,10,2021-12-30 17:42:20+00:00,Dogecoin,United States,NaN,NaN,USA,2021-12-30,2021,"(0.051605787, 0.7534284, 0.19496582)",0.051606,0.753428,0.194966,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25226,25226,25226,2021-01-02 18:31:29+00:00,dogecoin revolution has begun \n\n(this is not...,NaN,"Place(id='01a9a39529b27f36', fullName='Manhatt...",United States,USA,2021-01-02,2021,"(0.04810918, 0.6466202, 0.30527058)",0.048109,0.646620,0.305271,0,1,0
25246,25246,25246,2021-01-02 14:21:39+00:00,buy and hold dogecoin should be 3 in marketcap...,United States,NaN,NaN,USA,2021-01-02,2021,"(0.008810052, 0.17036851, 0.82082146)",0.008810,0.170369,0.820821,0,0,1
25250,25250,25250,2021-01-02 06:33:17+00:00,Join https://t.co/WtMANwe41B today and Avoid n...,Kenya,NaN,NaN,KEN,2021-01-02,2021,"(0.005783117, 0.23608427, 0.7581326)",0.005783,0.236084,0.758133,0,0,1
25254,25254,25254,2021-01-02 05:16:39+00:00,@elonmusk #dogecoin maybe 2021 is the year Dog...,United States,NaN,NaN,USA,2021-01-02,2021,"(0.0027663063, 0.2815225, 0.7157112)",0.002766,0.281523,0.715711,0,0,1


In [ ]:
agg_df = tweets_df.groupby(['Year','Country Code']).agg({'Negative_Count': 'sum', 'Neutral_Count': 'sum','Positive_Count': 'sum','Tweet':'count'})

In [ ]:
agg_df

Negative_Count  Neutral_Count  Positive_Count  Tweet
Year Country Code                                                      
2021 AND                        0              2               0      2
     ARE                        4             14              18     45
     ARG                        0             12               1     13
     AUS                       19             52              67    155
     AUT                        1              4               2      7
     BEL                        0              2               3      5
     BEN                        1              0               0      1
     BFA                        0              1               0      1
     BGR                        0              1               0      1
     BRA                        0             10               2     12
     CAN                        5             18              21     47
     CHE                        1              5               9     15
     CHN                        0              0               2      3
     COL                        2              2               0      4
     CZE                        0              1               0      1
     DEU                        7             44              47     98
     DNK                        1              1               2      4
     ESP                        1              3               0      5
     FIN                        2              0               5      7
     FRA                        0             12               6     18
     GBR                        4             34              72    113
     GEO                        1              5              13     19
     GHA                        0              2               0      2
     GNB                        0              1               0      1
     GRC                        0              1               1      2
     HUN                        0              3               3      6
     IDN                        0              1               1      2
     IND                       10             36              45     94
     IRL                        3              4               3     10
     IRQ                        0              1               3      4
     ISR                        0              0               1      1
     ITA                        1              3               4      8
     JPN                        3             26              47     81
     KEN                        0              1               4      5
     LKA                        0              0               3      3
     LTU                        0              1               0      1
     LVA                        0              0               1      1
     MEX                        1             16               4     24
     MLT                        0              1               0      1
     NLD                        3             19              18     41
     NOR                        1              2               2      5
     NZL                        5              4               8     17
     PAK                        2              2               7     11
     PER                        0              1               0      1
     PHL                        0              2               6      8
     POL                        0              1               1      2
     PRI                        0              0               2      4
     PRT                        0              2               3      7
     QAT                        0              1               0      1
     ROU                        1              1               0      2
     SAU                        0              1               2      3
     SGP                        2             12              11     26
     SVK                        0              1               0      1
     SWE                        0              2           

In [ ]:
agg_df.rename(columns={"Tweet": "Count"},inplace=True)

In [ ]:
agg_df['Negative_Percent'] = agg_df['Negative_Count']/agg_df['Count']
agg_df['Positive_Percent'] = agg_df['Positive_Count']/agg_df['Count']
agg_df['Neutral_Percent'] = agg_df['Neutral_Count']/agg_df['Count']

In [ ]:
agg_df['Negative_Percent'] = agg_df['Negative_Percent'].apply(lambda x: round(x,2))
agg_df['Positive_Percent'] = agg_df['Positive_Percent'].apply(lambda x: round(x,2))
agg_df['Neutral_Percent'] = agg_df['Neutral_Percent'].apply(lambda x: round(x,2))

In [ ]:
agg_df

Negative_Count  Neutral_Count  Positive_Count  Count  \
Year Country Code                                                         
2021 AND                        0              2               0      2   
     ARE                        4             14              18     45   
     ARG                        0             12               1     13   
     AUS                       19             52              67    155   
     AUT                        1              4               2      7   
     BEL                        0              2               3      5   
     BEN                        1              0               0      1   
     BFA                        0              1               0      1   
     BGR                        0              1               0      1   
     BRA                        0             10               2     12   
     CAN                        5             18              21     47   
     CHE                        1              5               9     15   
     CHN                        0              0               2      3   
     COL                        2              2               0      4   
     CZE                        0              1               0      1   
     DEU                        7             44              47     98   
     DNK                        1              1               2      4   
     ESP                        1              3               0      5   
     FIN                        2              0               5      7   
     FRA                        0             12               6     18   
     GBR                        4             34              72    113   
     GEO                        1              5              13     19   
     GHA                        0              2               0      2   
     GNB                        0              1               0      1   
     GRC                        0              1               1      2   
     HUN                        0              3               3      6   
     IDN                        0              1               1      2   
     IND                       10             36              45     94   
     IRL                        3              4               3     10   
     IRQ                        0              1               3      4   
     ISR                        0              0               1      1   
     ITA                        1              3               4      8   
     JPN                        3             26              47     81   
     KEN                        0              1               4      5   
     LKA                        0              0               3      3   
     LTU                        0              1               0      1   
     LVA                        0              0               1      1   
     MEX                        1             16               4     24   
     MLT                        0              1               0      1   
     NLD                        3             19              18     41   
     NOR                        1              2               2      5   
     NZL                        5              4               8     17   
     PAK                        2              2               7     11   
     PER                        0              1               0      1   
     PHL                        0              2               6      8   
     POL                        0              1               1      2   
     PRI                        0              0               2      4   
     PRT                        0              2               3      7   
     QAT                        0              1               0      1   
     ROU                        1              1               0      2   
     SAU                        0              1               2      3   
     SGP                        2           

In [ ]:
agg_df.to_csv('pradhan3.csv')

In [ ]:
agg_df1 = pd.read_csv("pradhan3.csv")

In [ ]:
agg_df1

,Year,Country Code,Negative_Count,Neutral_Count,Positive_Count,Count,Negative_Percent,Positive_Percent,Neutral_Percent
0,2021,AND,0,2,0,2,0.00,0.00,1.00
1,2021,ARE,4,14,18,45,0.09,0.40,0.31
2,2021,ARG,0,12,1,13,0.00,0.08,0.92
3,2021,AUS,19,52,67,155,0.12,0.43,0.34
4,2021,AUT,1,4,2,7,0.14,0.29,0.57
5,2021,BEL,0,2,3,5,0.00,0.60,0.40
6,2021,BEN,1,0,0,1,1.00,0.00,0.00
7,2021,BFA,0,1,0,1,0.00,0.00,1.00
8,2021,BGR,0,1,0,1,0.00,0.00,1.00
9,2021,BRA,0,10,2,12,0.00,0.17,0.83


In [ ]:
# Read in the GeoJSON file
with open('countries.geojson') as f:
    countries1 = json.load(f)

# Create an empty dictionary
# country_dict = {}

In [ ]:
sentiment_dict = {'positive': 0, 'negative': 0, 'neutral': 0}
# Loop through each feature in the GeoJSON file
for feature in countries1['features']:
    # Get the name of the country
    country_code = feature['properties']['ISO_A3']
    # print(country_code)
    # Assign a random number between 100-200 to the country
    if agg_df1.loc[agg_df1['Country Code'] == country_code].empty:
      continue
    else:  
      feature['properties']['tweet_count'] = str(agg_df1.loc[agg_df1['Country Code'] == country_code, 'Count'].iloc[0])
      sentiment_dict = {'positive': 0, 'negative': 0, 'neutral': 0}
      sentiment_dict['positive'] = agg_df1.loc[agg_df1['Country Code'] == country_code, 'Positive_Percent'].iloc[0]
      sentiment_dict['negative'] = agg_df1.loc[agg_df1['Country Code'] == country_code, 'Negative_Percent'].iloc[0]
      sentiment_dict['neutral'] = agg_df1.loc[agg_df1['Country Code'] == country_code, 'Neutral_Percent'].iloc[0]
      feature['properties']['sentiment'] = sentiment_dict

# ['Negative_Percent']
# Write out the modified GeoJSON data
with open('countries_count_sentiment_2021.geojson', 'w') as f:
    json.dump(countries1, f)

In [ ]:
# Read in the GeoJSON file
with open('/content/countries_count_sentiment_2021.geojson') as f:
    countries2 = json.load(f)

# Loop through each feature in the GeoJSON file
for i,feature in enumerate(countries2['features']):
    if i > 10:
      break
    # Get the name of the country
    country_name = feature['properties']['ISO_A3']
    print(feature)

{'type': 'Feature', 'properties': {'ADMIN': 'Aruba', 'ISO_A3': 'ABW'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-69.99693762899992, 12.577582098000036], [-69.93639075399994, 12.53172435100005], [-69.92467200399994, 12.519232489000046], [-69.91576087099992, 12.497015692000076], [-69.88019771999984, 12.453558661000045], [-69.87682044199994, 12.427394924000097], [-69.88809160099993, 12.417669989000046], [-69.90880286399994, 12.417792059000107], [-69.93053137899989, 12.425970770000035], [-69.94513912699992, 12.44037506700009], [-69.92467200399994, 12.44037506700009], [-69.92467200399994, 12.447211005000014], [-69.95856686099992, 12.463202216000099], [-70.02765865799992, 12.522935289000088], [-70.04808508999989, 12.53115469000008], [-70.05809485599988, 12.537176825000088], [-70.06240800699987, 12.546820380000057], [-70.06037350199995, 12.556952216000113], [-70.0510961579999, 12.574042059000064], [-70.04873613199993, 12.583726304000024], [-70.05264238199993, 12.600002346000053], [-7